In [2]:
import sys
sys.path.insert(0, '../')

import os
import torch
from functions import CustomSampler, load_pretrained_mlp, load_pretrained_cae, get_model_predictions, average_fusion_predictions, calculate_classification_report, calculate_fusion_accuracy
from ae_functions import get_latent_dataloader
from ae_models import improvedCAE, simpleCAE
from mlp_functions import train_mlp, validate_mlp, decision_level_fusion
from mlp_models import improvedMLP, simpleMLP
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np

In [3]:
def get_predictions(pred):
    batch_pred = []
    for batch in pred:
        batch_pred.extend(batch.argmax(dim=-1).cpu().numpy())
    return batch_pred

def unpack_labels(labels):
    batch_labels = []
    for batch in labels:
        batch_labels.extend(batch)
    return batch_labels

In [4]:
def get_confusion_matrix(preds_list, class_names, true_labels, setup, save_dir):
    sensor = ["Color", "Icub_left", "Icub_right"]
    for s, preds in enumerate(preds_list):
        matrix = confusion_matrix(true_labels, preds)

        # Reverse the order of class names
        class_names_reversed = class_names[::-1]

        # Reverse the order of the confusion matrix
        matrix_reversed = np.flip(matrix, axis=0)

        plt.figure()
        plt.imshow(matrix_reversed, cmap='Blues')

        # Add explicit values to the confusion matrix
        for i in range(matrix_reversed.shape[0]):
            for j in range(matrix_reversed.shape[1]):
                plt.text(j, i, str(matrix_reversed[i, j]), ha='center', va='center', color='orange')

        plt.colorbar()
        plt.xticks(range(len(class_names)), class_names)  # Set x-axis labels
        plt.yticks(range(len(class_names)), class_names_reversed)  # Set y-axis labels
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title(f'Confusion Matrix {setup} For {sensor[s]} Sensor')

        # Save the confusion matrix
        save_path = os.path.join(save_dir, f'confusion_matrix_{setup}_{sensor[s]}.png')
        plt.savefig(save_path)
        plt.close()
        print(f"Saved confusion matrix for {sensor[s]} sensor")

In [10]:
def get_accuracy(preds_list, true_labels):
    accuracies = []
    for preds in preds_list:
        accuracies.append(accuracy_score(true_labels, preds))
    print("color: ", accuracies[0], " left: ", accuracies[1])

In [8]:
BATCH_SIZE = 8

DEVICE = ""
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print("CUDA is available. Using GPU...")
else:
    DEVICE = torch.device("cpu")
    print("CUDA is not available. Using CPU...")

# Define the tool names and actions
TOOL_NAMES = ['hook', 'ruler', 'spatula', 'sshot']
ACTIONS = ['left_to_right', 'pull', 'push', 'right_to_left']

# All available object names
OBJECTS = ['0_woodenCube', '1_pearToy', '2_yogurtYellowbottle', '3_cowToy', '4_tennisBallYellowGreen',
            '5_blackCoinbag', '6_lemonSodaCan', '7_peperoneGreenToy', '8_boxEgg','9_pumpkinToy',
            '10_tomatoCan', '11_boxMilk', '12_containerNuts', '13_cornCob', '14_yellowFruitToy',
            '15_bottleNailPolisher', '16_boxRealSense', '17_clampOrange', '18_greenRectangleToy', '19_ketchupToy']

BASE_PATH = 'C:/Users/Frank/OneDrive/Bureaublad/ARC/deep-multimodal-learning/fusion'

sensor_color = "color"
sensor_left = "icub_left"
sensor_right = "icub_right"
sensor_depth = "depthcolormap"

# # Load CAE models
# model_path = "C:/Users/Frank/OneDrive/Bureaublad/ARC/deep-multimodal-learning/weights_ae/"
# cae_color = load_pretrained_cae(simpleCAE, model_path, "simple/simple_cae_ne3_b8_color.pth", DEVICE)
# cae_left = load_pretrained_cae(simpleCAE, model_path, "simple/simple_cae_ne3_b8_icub_left.pth", DEVICE)
# cae_right = load_pretrained_cae(simpleCAE, model_path, "simple/simple_cae_ne3_b8_icub_right.pth", DEVICE)
# cae_depth = load_pretrained_cae(simpleCAE, model_path, "simple/simple_cae_ne3_b8_depthcolormap.pth", DEVICE)

# # Load CAE models Improved
model_path = "C:/Users/Frank/OneDrive/Bureaublad/ARC/deep-multimodal-learning/weights_ae/"
cae_color = load_pretrained_cae(improvedCAE, model_path, "improved/improved_cae_ne3_b8_color_noise.pth", DEVICE)
cae_left = load_pretrained_cae(improvedCAE, model_path, "improved/improved_cae_ne3_b8_icub_left_noise.pth", DEVICE)
cae_right = load_pretrained_cae(improvedCAE, model_path, "improved/improved_cae_ne3_b8_icub_right_noise.pth", DEVICE)
#cae_depth = load_pretrained_cae(improvedCAE, model_path, "improved/improved_cae_ne3_b8_depthcolormap.pth", DEVICE)

# # Load CAE models Improved
# model_path = "C:/Users/Frank/OneDrive/Bureaublad/ARC/deep-multimodal-learning/weights_ae/"
# cae_color = load_pretrained_cae(improvedCAE, model_path, "improved/improved_cae_ne3_b8_color_noise.pth", DEVICE)
# cae_left = load_pretrained_cae(improvedCAE, model_path, "improved/improved_cae_ne3_b8_icub_left_noise.pth", DEVICE)
# cae_right = load_pretrained_cae(improvedCAE, model_path, "improved/improved_cae_ne3_b8_icub_right_noise.pth", DEVICE)
# # cae_depth = load_pretrained_cae(improvedCAE, model_path, "improved/improved_cae_ne3_b8_depth.pth", DEVICE)

# Assuming the datasets for all sensors are the same size
dataset_size = 640  # Replace with the actual size of your dataset
shuffled_indices = torch.randperm(dataset_size).tolist()

# Get dataloaders
set_name = "Testing"
test_loader_color, size = get_latent_dataloader(cae_color, BASE_PATH, OBJECTS, TOOL_NAMES, ACTIONS, sensor_color, set_name, labelc=0, shuffle=False, batch_size=BATCH_SIZE, device=DEVICE, indices=shuffled_indices)
test_loader_left, size = get_latent_dataloader(cae_left, BASE_PATH, OBJECTS, TOOL_NAMES, ACTIONS, sensor_left, set_name, labelc=0, shuffle=False, batch_size=BATCH_SIZE, device=DEVICE, indices=shuffled_indices)
#test_loader_right, size = get_latent_dataloader(cae_right, BASE_PATH, OBJECTS, TOOL_NAMES, ACTIONS, sensor_right, set_name, labelc=0, shuffle=False, batch_size=BATCH_SIZE, device=DEVICE, indices=shuffled_indices)
#test_loader_depth, size = get_latent_dataloader(cae_depth, BASE_PATH, OBJECTS, TOOL_NAMES, ACTIONS, sensor_depth, set_name, shuffle=False, batch_size=BATCH_SIZE, device=DEVICE, indices=shuffled_indices)

dataloaders = [test_loader_color, test_loader_left]

##### CONFIG
output_dim = 4
input_dim = size                                          

# Load MLP models
# model_path = "C:/Users/Frank/OneDrive/Bureaublad/ARC/deep-multimodal-learning/weights_mlp/"
# mlp_color = load_pretrained_mlp(simpleMLP, model_path, "simple/simple_mlp_ne3_b8_color_action.pth", DEVICE, input_dim, output_dim)
# mlp_left = load_pretrained_mlp(simpleMLP, model_path, "simple/simple_mlp_ne3_b8_icub_left_action.pth", DEVICE, input_dim, output_dim)
# mlp_right = load_pretrained_mlp(simpleMLP, model_path, "simple/simple_mlp_ne3_b8_icub_right_action.pth", DEVICE, input_dim, output_dim)
# mlp_depth = load_pretrained_mlp(simpleMLP, model_path, "simple/simple_mlp_ne3_b8_depthcolormap_action.pth", DEVICE, input_dim, output_dim)

# model_path = "C:/Users/Frank/OneDrive/Bureaublad/ARC/deep-multimodal-learning/weights_mlp/"
# mlp_color = load_pretrained_mlp(simpleMLP, model_path, "simple/simple_mlp_improved_cae_ne3_b8_color_action.pth", DEVICE, input_dim, output_dim)
# mlp_left = load_pretrained_mlp(simpleMLP, model_path, "simple/simple_mlp_improved_cae_ne3_b8_icub_left_action.pth", DEVICE, input_dim, output_dim)
# mlp_right = load_pretrained_mlp(simpleMLP, model_path, "simple/simple_mlp_improved_cae_ne3_b8_icub_right_action.pth", DEVICE, input_dim, output_dim)
# mlp_depth = load_pretrained_mlp(simpleMLP, model_path, "simple/simple_mlp_improved_cae_ne3_b8_depthcolormap_action.pth", DEVICE, input_dim, output_dim)

model_path = "C:/Users/Frank/OneDrive/Bureaublad/ARC/deep-multimodal-learning/weights_mlp/"
mlp_color = load_pretrained_mlp(improvedMLP, model_path, "simple/improved_mlp_improved_cae_ne3_b8_color_tool.pth", DEVICE, input_dim, output_dim)
mlp_left = load_pretrained_mlp(improvedMLP, model_path, "simple/improved_mlp_improved_cae_ne3_b8_icub_left_tool.pth", DEVICE, input_dim, output_dim)
#mlp_right = load_pretrained_mlp(simpleMLP, model_path, "simple/simple_mlp_improved_cae_ne3_b8_icub_right_noise.pth", DEVICE, input_dim, output_dim)

# model_path = "C:/Users/Frank/OneDrive/Bureaublad/ARC/deep-multimodal-learning/weights_mlp/"
# mlp_color = load_pretrained_mlp(improvedMLP, model_path, "simple/improved_mlp_simple_cae_ne3_b8_color.pth", DEVICE, input_dim, output_dim)
# mlp_left = load_pretrained_mlp(improvedMLP, model_path, "simple/improved_mlp_simple_cae_ne3_b8_icub_left.pth", DEVICE, input_dim, output_dim)
# mlp_right = load_pretrained_mlp(improvedMLP, model_path, "simple/improved_mlp_simple_cae_ne3_b8_icub_right.pth", DEVICE, input_dim, output_dim)
# mlp_depth = load_pretrained_mlp(improvedMLP, model_path, "simple/improved_mlp_simple_cae_ne3_b8_depthcolormap.pth", DEVICE, input_dim, output_dim)

# model_path = "C:/Users/Frank/OneDrive/Bureaublad/ARC/deep-multimodal-learning/weights_mlp/"
# mlp_color = load_pretrained_mlp(improvedMLP, model_path, "simple/improved_mlp_improved_cae_ne3_b8_color_action.pth", DEVICE, input_dim, output_dim)
# mlp_left = load_pretrained_mlp(improvedMLP, model_path, "simple/improved_mlp_improved_cae_ne3_b8_icub_left_action.pth", DEVICE, input_dim, output_dim)
# mlp_right = load_pretrained_mlp(improvedMLP, model_path, "simple/improved_mlp_improved_cae_ne3_b8_icub_right_action.pth", DEVICE, input_dim, output_dim)
# mlp_depth = load_pretrained_mlp(improvedMLP, model_path, "simple/improved_mlp_improved_cae_ne3_b8_depthcolormap_action.pth", DEVICE, input_dim, output_dim)

models = [mlp_color, mlp_left]

# Get predictions
model_predictions = [get_model_predictions(model, dataloader, DEVICE) for model, dataloader in zip(models, dataloaders)]

color = model_predictions[0]
left = model_predictions[1]
#right = model_predictions[2]
#depth = model_predictions[3]

# Gather true labels and calculate accuracy
# class_names = ['left_to_right', 'pull', 'push', 'right_to_left']
class_names = ['hook', 'ruler', 'spatula', 'sshot']
true_labels = [labels.cpu().numpy() for _, labels in test_loader_color]
true_labels = unpack_labels(true_labels)

color_preds = get_predictions(color)
left_preds = get_predictions(left)
#right_preds = get_predictions(right)
#depth_preds = get_predictions(depth)

CUDA is available. Using GPU...


In [11]:
get_accuracy([color_preds, left_preds], true_labels)

color:  0.209375  left:  0.25


In [104]:
dir = "C:/Users/Frank/OneDrive/Bureaublad/ARC/deep-multimodal-learning/models/runs/results/matrixes/"
get_confusion_matrix([color_preds, left_preds], class_names, true_labels, "improvedCAE+improvedMLP+Tool", dir)

Saved confusion matrix for Color sensor
Saved confusion matrix for Icub_left sensor
Saved confusion matrix for Icub_right sensor
